In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
from DeepLearning.Project1.data import *

In [2]:
# File based on https://github.com/huyvnphan/PyTorch_CIFAR10/
class VGG(nn.Module):
    def __init__(self, features, num_classes=10, avgpool_size=(1,1), add_dropout=True):
        super(VGG, self).__init__()
        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(avgpool_size)
        if add_dropout:
            self.classifier = nn.Sequential(
                nn.Linear(512 * avgpool_size[0] * avgpool_size[1], 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, num_classes),
            )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(512 * avgpool_size[0] * avgpool_size[1], 4096),
                nn.ReLU(True),
                nn.Linear(4096, 4096),
                nn.ReLU(True),
                nn.Linear(4096, num_classes),
            )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

 # Create convolution part of VGG11_bn archhitecture
def make_vgg11_bn_layers(cfg = None):
    if cfg == None:
        cfg = [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"]
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

 # Create VGG11_bn model
def vgg11_bn(device="cpu", num_classes=10):
    model = VGG(make_vgg11_bn_layers(), num_classes=num_classes)
    return model

vgg_cfg = {
    '8':  [64, 'M', 128, 'M', 256, 'M', 512, 'M', 512, 'M'],
    '11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    '13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    '16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    '19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}



def eval_accuracy(model, dataloader, training_device='cpu'):
    model.eval()
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(training_device), labels.to(training_device)
            pred = torch.argmax(model(inputs), dim=1)

            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    model.train()
    return correct/all_so_far

In [3]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [4]:
def train_one_epoch(model, optimizer, criterion, dataloader_train, training_device):
    model.train()
    for inputs, labels in dataloader_train:
        inputs, labels = inputs.to(training_device), labels.to(training_device)
        optimizer.zero_grad()
        loss = criterion(model(inputs), labels)
        loss.backward()
        optimizer.step()

def train_one_run(model, optimizer, criterion,
                  dataloader_train, dataloader_val,
                  max_epochs, early_stopper, 
                  trajectory, 
                  training_device='cuda', mem_bg_allocated=0, mem_bg_reserved=0,
                  *_args, **_kwargs):
    model.train()
    model.to(training_device)

    for epoch in range(max_epochs):
        start_time = time.time()
        
        train_one_epoch(model, optimizer, criterion, dataloader_train, training_device)
        
        end_time = time.time()
        
        training_accuracy = eval_accuracy(model, dataloader_train, training_device)
        validation_accuracy = eval_accuracy(model, dataloader_val, training_device)
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "train": training_accuracy,
            "validation": validation_accuracy,
            "start_time": start_time,
            "duration": end_time - start_time,
            "memory_allocated_mb": (torch.cuda.memory_allocated() - mem_bg_allocated)/1024/1024,
            "memory_reserved_mb": (torch.cuda.memory_reserved() - mem_bg_reserved)/1024/1024,
        })
        
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            return early_stopper.best_backup
    
    return model

In [5]:
def run_train_experiment(arch_name, model_factory, aug_name, aug_factory, train_name, train_func, run):
    path = f"experiments/train_{train_name}_aug_{aug_name}_arch_{arch_name}_{run}_"
    
    # 
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(aug_factory, model_factory)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": arch_name,
                "run": run,
                "augment": aug_name,
                "train": train_name, 
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [6]:
def standard(aug_factory, model_factory):
    torch.cuda.empty_cache()
    mem_bg_allocated = torch.cuda.memory_allocated()
    mem_bg_reserved = torch.cuda.memory_reserved()
    
    train, test, val = aug_factory()
    model = model_factory()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    early_stopper = EarlyStopper(patience = 5)
    trajectory = []
    model = train_one_run(model, optimizer, criterion,
                          train, val,
                          200, early_stopper,
                          trajectory, 
                          mem_bg_allocated=mem_bg_allocated,
                          mem_bg_reserved=mem_bg_reserved)
    validation_accuracy = eval_accuracy(model, val, "cuda")
    return model, trajectory, validation_accuracy

def weightdecay(aug_factory, model_factory):
    torch.cuda.empty_cache()
    mem_bg_allocated = torch.cuda.memory_allocated()
    mem_bg_reserved = torch.cuda.memory_reserved()
    
    train, test, val = aug_factory()
    model = model_factory()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    early_stopper = EarlyStopper(patience = 5)
    trajectory = []
    model = train_one_run(model, optimizer, criterion,
                          train, val,
                          200, early_stopper,
                          trajectory, 
                          mem_bg_allocated=mem_bg_allocated,
                          mem_bg_reserved=mem_bg_reserved)
    validation_accuracy = eval_accuracy(model, val, "cuda")
    return model, trajectory, validation_accuracy

In [7]:
def flip(bs=128):
    return augmented_cifar10_dataset_randomflip(bs=bs)

def smallrotate(bs=128):
    return augmented_cifar10_dataset_rotate_randomapply(5, bs=bs)

def fliprotate(bs=128):
    return augmented_cifar10_dataset_randomflip_rotate_randomapply(5, bs=bs)


In [13]:
def vgg_from_cfg(cfg_key, *a, **kw):
    return VGG(
        make_vgg11_bn_layers(cfg=vgg_cfg[cfg_key]),
        num_classes=10,
        avgpool_size=(1,1),
        *a,
        **kw
    ).to("cuda")

def vgg13():
    return vgg_from_cfg("13")

def vgg16():
    return vgg_from_cfg("16")

def vgg13nodropout():
    return vgg_from_cfg("13", add_dropout=False)

def vgg16nodropout():
    return vgg_from_cfg("16", add_dropout=False)

In [14]:
experiment_list = [
    (
        archfactory.__name__, 
        archfactory, 
        augfactory.__name__, 
        augfactory, 
        trainfunc.__name__,
        trainfunc,
        str(run)
    )
    for run in range(1, 10) 
    for archfactory in [vgg13nodropout, vgg16nodropout]
    for augfactory in [flip]
    for trainfunc in [standard]
] + [
    (
        archfactory.__name__, 
        archfactory, 
        augfactory.__name__, 
        augfactory, 
        trainfunc.__name__,
        trainfunc,
        str(run)
    )
    for run in range(1, 10) 
    for archfactory in [vgg13, vgg16]
    for augfactory in [flip]
    for trainfunc in [weightdecay]
]

In [15]:
len(experiment_list)

36

In [16]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        *experiment[::2]
    )
    try:
        run_train_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-03-22T08:21:30.361008 vgg13nodropout flip standard 1
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.3846
Epoch: 1, Accuracy on validation set: 0.5738
Epoch: 2, Accuracy on validation set: 0.654
Epoch: 3, Accuracy on validation set: 0.6906
Epoch: 4, Accuracy on validation set: 0.7424
Epoch: 5, Accuracy on validation set: 0.7684
Epoch: 6, Accuracy on validation set: 0.7912
Epoch: 7, Accuracy on validation set: 0.8116
Epoch: 8, Accuracy on validation set: 0.763
Epoch: 9, Accuracy on validation set: 0.7854
Epoch: 10, Accuracy on validation set: 0.8008
Epoch: 11, Accuracy on validation set: 0.8284
Epoch: 12, Accuracy on validation set: 0.8116
Epoch: 13, Accuracy on validation set: 0.8356
Epoch: 14, Accuracy on validation set: 0.8356
Epoch: 15, Accuracy on validation set: 0.8466
Epoch: 16, Accuracy on validation set: 0.8442
Epoch: 17, Accuracy on validation set: 0.85
Epoch: 18, Accuracy on validation set: 0.8458


Epoch: 17, Accuracy on validation set: 0.8364
Epoch: 18, Accuracy on validation set: 0.788
Epoch: 19, Accuracy on validation set: 0.841
Epoch: 20, Accuracy on validation set: 0.832
Epoch: 21, Accuracy on validation set: 0.8392
Early stop
Time: 2023-03-22T09:30:22.074245 vgg13nodropout flip standard 4
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.3402
Epoch: 1, Accuracy on validation set: 0.543
Epoch: 2, Accuracy on validation set: 0.6488
Epoch: 3, Accuracy on validation set: 0.7072
Epoch: 4, Accuracy on validation set: 0.7102
Epoch: 5, Accuracy on validation set: 0.7874
Epoch: 6, Accuracy on validation set: 0.7412
Epoch: 7, Accuracy on validation set: 0.7634
Epoch: 8, Accuracy on validation set: 0.7646
Epoch: 9, Accuracy on validation set: 0.7636
Epoch: 10, Accuracy on validation set: 0.8068
Epoch: 11, Accuracy on validation set: 0.8322
Epoch: 12, Accuracy on validation set: 0.7852
Epoch: 13, Accuracy on validation s

Epoch: 19, Accuracy on validation set: 0.8518
Epoch: 20, Accuracy on validation set: 0.8646
Epoch: 21, Accuracy on validation set: 0.862
Epoch: 22, Accuracy on validation set: 0.8702
Epoch: 23, Accuracy on validation set: 0.8582
Epoch: 24, Accuracy on validation set: 0.8588
Epoch: 25, Accuracy on validation set: 0.8632
Epoch: 26, Accuracy on validation set: 0.8706
Epoch: 27, Accuracy on validation set: 0.8654
Epoch: 28, Accuracy on validation set: 0.874
Epoch: 29, Accuracy on validation set: 0.8634
Epoch: 30, Accuracy on validation set: 0.8722
Epoch: 31, Accuracy on validation set: 0.8668
Epoch: 32, Accuracy on validation set: 0.8814
Epoch: 33, Accuracy on validation set: 0.8682
Epoch: 34, Accuracy on validation set: 0.8674
Epoch: 35, Accuracy on validation set: 0.8754
Epoch: 36, Accuracy on validation set: 0.8642
Epoch: 37, Accuracy on validation set: 0.8766
Early stop
Time: 2023-03-22T10:43:58.515983 vgg13nodropout flip standard 7
Files already downloaded and verified
Files already d

Epoch: 3, Accuracy on validation set: 0.6678
Epoch: 4, Accuracy on validation set: 0.7154
Epoch: 5, Accuracy on validation set: 0.7242
Epoch: 6, Accuracy on validation set: 0.7608
Epoch: 7, Accuracy on validation set: 0.7108
Epoch: 8, Accuracy on validation set: 0.7714
Epoch: 9, Accuracy on validation set: 0.7898
Epoch: 10, Accuracy on validation set: 0.79
Epoch: 11, Accuracy on validation set: 0.774
Epoch: 12, Accuracy on validation set: 0.8264
Epoch: 13, Accuracy on validation set: 0.8162
Epoch: 14, Accuracy on validation set: 0.7998
Epoch: 15, Accuracy on validation set: 0.8316
Epoch: 16, Accuracy on validation set: 0.8312
Epoch: 17, Accuracy on validation set: 0.8418
Epoch: 18, Accuracy on validation set: 0.8552
Epoch: 19, Accuracy on validation set: 0.8292
Epoch: 20, Accuracy on validation set: 0.854
Epoch: 21, Accuracy on validation set: 0.856
Epoch: 22, Accuracy on validation set: 0.8478
Epoch: 23, Accuracy on validation set: 0.8524
Epoch: 24, Accuracy on validation set: 0.8662


Epoch: 32, Accuracy on validation set: 0.8398
Epoch: 33, Accuracy on validation set: 0.8368
Epoch: 34, Accuracy on validation set: 0.8572
Epoch: 35, Accuracy on validation set: 0.8414
Early stop
Time: 2023-03-22T12:54:38.399787 vgg16 flip weightdecay 3
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.2744
Epoch: 1, Accuracy on validation set: 0.4454
Epoch: 2, Accuracy on validation set: 0.4266
Epoch: 3, Accuracy on validation set: 0.552
Epoch: 4, Accuracy on validation set: 0.6572
Epoch: 5, Accuracy on validation set: 0.6612
Epoch: 6, Accuracy on validation set: 0.7282
Epoch: 7, Accuracy on validation set: 0.7582
Epoch: 8, Accuracy on validation set: 0.7736
Epoch: 9, Accuracy on validation set: 0.7722
Epoch: 10, Accuracy on validation set: 0.7904
Epoch: 11, Accuracy on validation set: 0.7968
Epoch: 12, Accuracy on validation set: 0.8214
Epoch: 13, Accuracy on validation set: 0.7912
Epoch: 14, Accuracy on validation set:

Epoch: 3, Accuracy on validation set: 0.6112
Epoch: 4, Accuracy on validation set: 0.6274
Epoch: 5, Accuracy on validation set: 0.6804
Epoch: 6, Accuracy on validation set: 0.6988
Epoch: 7, Accuracy on validation set: 0.7456
Epoch: 8, Accuracy on validation set: 0.7648
Epoch: 9, Accuracy on validation set: 0.7904
Epoch: 10, Accuracy on validation set: 0.8082
Epoch: 11, Accuracy on validation set: 0.7952
Epoch: 12, Accuracy on validation set: 0.8228
Epoch: 13, Accuracy on validation set: 0.8342
Epoch: 14, Accuracy on validation set: 0.8194
Epoch: 15, Accuracy on validation set: 0.8346
Epoch: 16, Accuracy on validation set: 0.8426
Epoch: 17, Accuracy on validation set: 0.8418
Epoch: 18, Accuracy on validation set: 0.8268
Epoch: 19, Accuracy on validation set: 0.8536
Epoch: 20, Accuracy on validation set: 0.8616
Epoch: 21, Accuracy on validation set: 0.8632
Epoch: 22, Accuracy on validation set: 0.8386
Epoch: 23, Accuracy on validation set: 0.8482
Epoch: 24, Accuracy on validation set: 0.

Epoch: 8, Accuracy on validation set: 0.776
Epoch: 9, Accuracy on validation set: 0.7512
Epoch: 10, Accuracy on validation set: 0.8018
Epoch: 11, Accuracy on validation set: 0.789
Epoch: 12, Accuracy on validation set: 0.8092
Epoch: 13, Accuracy on validation set: 0.8248
Epoch: 14, Accuracy on validation set: 0.828
Epoch: 15, Accuracy on validation set: 0.823
Epoch: 16, Accuracy on validation set: 0.8402
Epoch: 17, Accuracy on validation set: 0.8324
Epoch: 18, Accuracy on validation set: 0.8332
Epoch: 19, Accuracy on validation set: 0.7814
Epoch: 20, Accuracy on validation set: 0.8298
Epoch: 21, Accuracy on validation set: 0.8428
Epoch: 22, Accuracy on validation set: 0.8558
Epoch: 23, Accuracy on validation set: 0.865
Epoch: 24, Accuracy on validation set: 0.8442
Epoch: 25, Accuracy on validation set: 0.8492
Epoch: 26, Accuracy on validation set: 0.8672
Epoch: 27, Accuracy on validation set: 0.8536
Epoch: 28, Accuracy on validation set: 0.8658
Epoch: 29, Accuracy on validation set: 0.